In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../dp_common'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
from ipynb.fs.full.common_apps import get_spark_session
from ipynb.fs.full.common_apps import get_snowflake_options
from ipynb.fs.full.common_apps import get_snowflake_source

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
A = '''select *, case
WHEN TXN_HR < 8 THEN '0-8'
WHEN TXN_HR >=8 AND TXN_HR <16 THEN '8-16'
WHEN TXN_HR >=16 AND TXN_HR < 24 THEN '16-24' END AS TIME_BUCKET
FROM(select a.user_id, a.USER_PROFILE_TYPE, a.transaction_time_gmt,date(a.transaction_time_gmt) as txn_date, extract(hour from a.TRANSACTION_TIME_GMT) as txn_hr,extract(day from a.TRANSACTION_TIME_GMT) as day,extract(month from a.TRANSACTION_TIME_GMT) as month,
a.CARD_PROGRAM, a.REQUESTED_AMOUNT, a.REQUESTED_AMOUNT_CURRENCY, a.TRANSACTION_AMOUNT,
(cast((b.credit_limit - a.AVAILABLE_BALANCE) as float)/cast(b.CREDIT_LIMIT as float))*100 as cc_utilised_prcnt,
a.mcc, a.PAN_ENTRY_MODE, a.TRANSACTION_ID, a.ORIGINAL_TRANSACTION_ID, b.USER_BUREAU_STATUS, b.CREDIT_LIMIT,
(date(getdate()) - date(b.CREDIT_CARD_CREATED_TIME)) as cc_age
from zolve.facts.OVERALL_TRANSACTIONS a
left join zolve.facts.USER_ONBOARDING_STAGES b
on a.user_id = b.user_id
where a.transaction_status = 'SUCCESS' and a.PRODUCT_TYPE = 'Credit Card' and a.IS_INTERNAL_TRANSACTION = 'no' and b.user_category <> 'fnf' and a.TRANSACTION_AMOUNT <> 0
and a.TRANSACTION_SUB_STATUS <> 'reversal' and a.transaction_type = 'debit' AND a.TRANSACTION_TYPE_CODE in ('00'))
'''

In [ ]:
spark = get_spark_session("Anomaly Detection", swan_spark_conf)

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

try:
    data_frame = spark.read.format(get_snowflake_source()) \
    .options(**get_snowflake_options()) \
    .option("query", A) \
    .load()
    df = data_frame.toPandas()
except Exception as exp:
    print(exp)
spark.stop()
df.head()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
net.snowflake#snowflake-jdbc added as a dependency
net.snowflake#spark-snowflake_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f9f50a7a-4455-4718-a2bf-f3e3263922f9;1.0
	confs: [default]
	found net.snowflake#snowflake-jdbc;3.12.17 in central
	found net.snowflake#spark-snowflake_2.12;2.8.4-spark_3.0 in central
	found net.snowflake#snowflake-ingest-sdk;0.10.1 in central
:: resolution report :: resolve 489ms :: artifacts dl 6ms
	:: modules in use:
	net.snowflake#snowflake-ingest-sdk;0.10.1 from central in [default]
	net.snowflake#snowflake-jdbc;3.12.17 from central in [default]
	net.snowflake#spark-snowflake_2.12;2.8.4-spark_3.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted||

,USER_ID,USER_PROFILE_TYPE,TRANSACTION_TIME_GMT,TXN_DATE,TXN_HR,DAY,MONTH,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,TRANSACTION_AMOUNT,CC_UTILISED_PRCNT,MCC,PAN_ENTRY_MODE,TRANSACTION_ID,ORIGINAL_TRANSACTION_ID,USER_BUREAU_STATUS,CREDIT_LIMIT,CC_AGE,TIME_BUCKET
0,7f442e27-16ed-457a-9af3-a4654727fe88,STUDENT,2022-03-11 07:11:21,2022-03-11,7,11,3,physical_credit_card,39.95,USD,39.95,4.551111,5967,162,19296082,19164218,NTC,1800.0,235,0-8
1,7f442e27-16ed-457a-9af3-a4654727fe88,STUDENT,2021-11-04 04:49:04,2021-11-04,4,4,11,physical_credit_card,31.64,USD,31.64,83.388333,5968,292,3903591,3891447,NTC,1800.0,235,0-8
2,7f442e27-16ed-457a-9af3-a4654727fe88,STUDENT,2021-10-09 04:23:21,2021-10-09,4,9,10,physical_credit_card,84.39,USD,84.39,39.857778,5942,160,2950008,2846049,NTC,1800.0,235,0-8
3,7f442e27-16ed-457a-9af3-a4654727fe88,STUDENT,2021-09-29 06:31:12,2021-09-29,6,29,9,physical_credit_card,38.99,USD,38.99,27.240000,5942,160,2434991,2406626,NTC,1800.0,235,0-8
4,7f442e27-16ed-457a-9af3-a4654727fe88,STUDENT,2022-01-18 04:25:54,2022-01-18,4,18,1,physical_credit_card,12.65,USD,12.65,7.949444,5942,160,12973306,12725146,NTC,1800.0,235,0-8


In [ ]:
df1 = df.drop_duplicates()

In [ ]:
import datetime as dt
df1['TRANSACTION_TIME_GMT'] = pd.to_datetime(df1['TRANSACTION_TIME_GMT'])
df1['TXN_TIME'] = df1['TRANSACTION_TIME_GMT'].dt.time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df2 = df1.groupby(['USER_ID','TRANSACTION_ID'])['TRANSACTION_ID'].count().reset_index(name = 'COUNT')

In [ ]:
df3 = df1.merge(df2, on = ['USER_ID','TRANSACTION_ID'])

In [ ]:
df3 = df3.sort_values(by = ['USER_ID', 'DAY', 'MONTH', 'TXN_TIME'])

In [ ]:
df3['COUNT_24HR'] = df3.groupby(['USER_ID', 'DAY','MONTH'])['COUNT'].cumsum(axis=0)

In [ ]:
df3['AMT_24HR'] = df3.groupby(['USER_ID', 'DAY', 'MONTH'])['TRANSACTION_AMOUNT'].cumsum(axis=0)

In [ ]:
df3['COUNT_TIME_BUCKET'] = df3.groupby(['USER_ID', 'DAY', 'MONTH','TIME_BUCKET'])['COUNT'].cumsum(axis=0)

In [ ]:
df3['WEEK_DAY']  = df3['TRANSACTION_TIME_GMT'].dt.day_name()

In [ ]:
# df3.drop(['TRANSACTION_TIME_GMT','COUNT','TXN_TIME','MONTH','DAY'],axis=1, inplace=True)

In [ ]:
df3['TOTAL_COUNTS'] = df3.groupby(['USER_ID'])['COUNT'].cumsum(axis=0)

In [ ]:
df3['TOTAL_AMOUNT'] = df3.groupby(['USER_ID'])['TRANSACTION_AMOUNT'].cumsum(axis=0)

In [ ]:
df4 = df3.groupby(['USER_ID','TXN_DATE'])['USER_ID'].count()

In [ ]:
df4

USER_ID                               TXN_DATE  
0002be66-d809-4ca1-a5a0-4a522adc9323  2022-04-08    1
                                      2022-04-09    2
                                      2022-04-10    1
                                      2022-04-11    1
                                      2022-04-21    1
                                                   ..
fffc5865-26fe-4186-a63a-96587d5071d4  2022-03-25    2
                                      2022-03-26    1
                                      2022-03-28    1
                                      2022-04-18    2
                                      2022-04-24    1
Name: USER_ID, Length: 296308, dtype: int64

In [ ]:
df3[df3['USER_ID']=='0002be66-d809-4ca1-a5a0-4a522adc9323']

,USER_ID,USER_PROFILE_TYPE,TRANSACTION_TIME_GMT,TXN_DATE,TXN_HR,DAY,MONTH,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,...,CC_AGE,TIME_BUCKET,TXN_TIME,COUNT,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY,TOTAL_COUNTS,TOTAL_AMOUNT
470568,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-08 19:12:02,2022-04-08,19,8,4,virtual_credit_card,26.89,USD,...,20,16-24,19:12:02,1,1,26.89,1,Friday,1,26.89
470569,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-09 00:47:00,2022-04-09,0,9,4,virtual_credit_card,93.93,USD,...,20,0-8,00:47:00,1,1,93.93,1,Saturday,2,120.82
470571,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-09 06:32:30,2022-04-09,6,9,4,physical_credit_card,1703.53,USD,...,20,0-8,06:32:30,1,2,1797.46,2,Saturday,3,1824.35
470572,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-10 17:55:13,2022-04-10,17,10,4,virtual_credit_card,5.00,USD,...,20,16-24,17:55:13,1,1,5.00,1,Sunday,4,1829.35
470567,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-11 03:51:13,2022-04-11,3,11,4,physical_credit_card,14.31,USD,...,20,0-8,03:51:13,1,1,14.31,1,Monday,5,1843.66
470570,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-21 17:01:04,2022-04-21,17,21,4,physical_credit_card,43.29,USD,...,20,16-24,17:01:04,1,1,43.29,1,Thursday,6,1886.95


In [ ]:
df3.head()

,USER_ID,USER_PROFILE_TYPE,TRANSACTION_TIME_GMT,TXN_DATE,TXN_HR,DAY,MONTH,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,...,TIME_BUCKET,TXN_TIME,COUNT,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY,total_counts,TOTAL_COUNTS,TOTAL_AMOUNT
269994,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-08 19:12:02,2022-04-08,19,8,4,virtual_credit_card,26.89,USD,...,16-24,19:12:02,1,1,26.89,1,Friday,1,1,26.89
269995,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-09 00:47:00,2022-04-09,0,9,4,virtual_credit_card,93.93,USD,...,0-8,00:47:00,1,1,93.93,1,Saturday,2,2,120.82
269996,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-09 06:32:30,2022-04-09,6,9,4,physical_credit_card,1703.53,USD,...,0-8,06:32:30,1,2,1797.46,2,Saturday,3,3,1824.35
269997,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-10 17:55:13,2022-04-10,17,10,4,virtual_credit_card,5.00,USD,...,16-24,17:55:13,1,1,5.00,1,Sunday,4,4,1829.35
269993,0002be66-d809-4ca1-a5a0-4a522adc9323,STUDENT,2022-04-11 03:51:13,2022-04-11,3,11,4,physical_credit_card,14.31,USD,...,0-8,03:51:13,1,1,14.31,1,Monday,5,5,1843.66


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df3['CARD_PROGRAM'] = le.fit_transform(df3['CARD_PROGRAM'])
df3['USER_PROFILE_TYPE'] = le.fit_transform(df3['USER_PROFILE_TYPE'])
df3['REQUESTED_AMOUNT_CURRENCY'] = le.fit_transform(df3['REQUESTED_AMOUNT_CURRENCY'])
df3['USER_BUREAU_STATUS'] = le.fit_transform(df3['USER_BUREAU_STATUS'])
df3['TIME_BUCKET'] = le.fit_transform(df3['TIME_BUCKET'])
df3['WEEK_DAY'] = le.fit_transform(df3['WEEK_DAY'])

In [ ]:
df3['PAN_ENTRY_MODE'] = df3['PAN_ENTRY_MODE'].replace('***', '000')

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475338 entries, 144967 to 38542
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   USER_ID                    475338 non-null  object 
 1   USER_PROFILE_TYPE          475338 non-null  int64  
 2   TXN_DATE                   475338 non-null  object 
 3   TXN_HR                     475338 non-null  object 
 4   CARD_PROGRAM               475338 non-null  int64  
 5   REQUESTED_AMOUNT           475338 non-null  float64
 6   REQUESTED_AMOUNT_CURRENCY  475338 non-null  int64  
 7   TRANSACTION_AMOUNT         475338 non-null  float64
 8   CC_UTILISED_PRCNT          475227 non-null  float64
 9   MCC                        475338 non-null  object 
 10  PAN_ENTRY_MODE             475338 non-null  object 
 11  TRANSACTION_ID             475338 non-null  object 
 12  ORIGINAL_TRANSACTION_ID    454961 non-null  object 
 13  USER_BUREAU_STATUS       

In [ ]:
df3['TXN_HR'] = df3['TXN_HR'].astype(int)
df3['MCC'] = df3['MCC'].astype(int)
df3['PAN_ENTRY_MODE'] = df3['PAN_ENTRY_MODE'].astype(int)
df3['CC_AGE'] = df3['CC_AGE'].astype(int)

In [ ]:
features = pd.DataFrame(index=df3.index)

numerical_columns = ['USER_PROFILE_TYPE', 'TXN_HR', 'CARD_PROGRAM','REQUESTED_AMOUNT', 'REQUESTED_AMOUNT_CURRENCY',
       'TRANSACTION_AMOUNT','CC_UTILISED_PRCNT', 'MCC', 'PAN_ENTRY_MODE','USER_BUREAU_STATUS', 'CREDIT_LIMIT',
       'CC_AGE', 'TIME_BUCKET', 'COUNT_24HR', 'AMT_24HR', 'COUNT_TIME_BUCKET','WEEK_DAY']
features[numerical_columns] = df3[numerical_columns]

In [ ]:
features.head()

,USER_PROFILE_TYPE,TXN_HR,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,TRANSACTION_AMOUNT,CC_UTILISED_PRCNT,MCC,PAN_ENTRY_MODE,USER_BUREAU_STATUS,CREDIT_LIMIT,CC_AGE,TIME_BUCKET,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY
144967,0,19,2,26.89,31,26.89,86.873810,5812,290,2,2100.0,13,1,1,26.89,1,0
144968,0,0,2,93.93,31,93.93,86.873810,5651,70,2,2100.0,13,0,1,93.93,1,2
144969,0,6,1,1703.53,31,1703.53,87.793333,3026,160,2,2100.0,13,0,2,1797.46,2,2
144970,0,17,2,5.00,31,5.00,87.111905,7210,162,2,2100.0,13,1,1,5.00,1,3
144966,0,3,1,14.31,31,14.31,87.793333,5815,290,2,2100.0,13,0,1,14.31,1,1


In [ ]:
features.dropna(how='any', inplace=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475227 entries, 144967 to 38542
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   USER_PROFILE_TYPE          475227 non-null  int64  
 1   TXN_HR                     475227 non-null  int64  
 2   CARD_PROGRAM               475227 non-null  int64  
 3   REQUESTED_AMOUNT           475227 non-null  float64
 4   REQUESTED_AMOUNT_CURRENCY  475227 non-null  int64  
 5   TRANSACTION_AMOUNT         475227 non-null  float64
 6   CC_UTILISED_PRCNT          475227 non-null  float64
 7   MCC                        475227 non-null  int64  
 8   PAN_ENTRY_MODE             475227 non-null  int64  
 9   USER_BUREAU_STATUS         475227 non-null  int64  
 10  CREDIT_LIMIT               475227 non-null  float64
 11  CC_AGE                     475227 non-null  int64  
 12  TIME_BUCKET                475227 non-null  int64  
 13  COUNT_24HR               

In [ ]:
# to check rows if any null values are there
# features[features.isna().any(axis=1)]

In [ ]:
from sklearn.ensemble import IsolationForest
forest = IsolationForest(random_state= 1, n_estimators= 100, contamination = float(0.01), max_samples= 'auto')
forest.fit(features)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.01, random_state=1)

In [ ]:
scores = forest.score_samples(features)

In [ ]:
df3.dropna(subset = ['CC_UTILISED_PRCNT'], inplace=True)

In [ ]:
df3['anamoly_scores'] = scores

In [ ]:
df3['anomaly'] = forest.predict(features)
df3.head()

,USER_ID,USER_PROFILE_TYPE,TXN_DATE,TXN_HR,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,TRANSACTION_AMOUNT,CC_UTILISED_PRCNT,MCC,...,USER_BUREAU_STATUS,CREDIT_LIMIT,CC_AGE,TIME_BUCKET,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY,anamoly_scores,anomaly
144967,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-08,19,2,26.89,31,26.89,86.873810,5812,...,2,2100.0,13,1,1,26.89,1,0,-0.508667,1
144968,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-09,0,2,93.93,31,93.93,86.873810,5651,...,2,2100.0,13,0,1,93.93,1,2,-0.521941,1
144969,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-09,6,1,1703.53,31,1703.53,87.793333,3026,...,2,2100.0,13,0,2,1797.46,2,2,-0.672711,-1
144970,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-10,17,2,5.00,31,5.00,87.111905,7210,...,2,2100.0,13,1,1,5.00,1,3,-0.530777,1
144966,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-11,3,1,14.31,31,14.31,87.793333,5815,...,2,2100.0,13,0,1,14.31,1,1,-0.416860,1


In [ ]:
df3['anomaly'].value_counts()

 1    470474
-1      4753
Name: anomaly, dtype: int64

In [ ]:
df3.head()

,USER_ID,USER_PROFILE_TYPE,TXN_DATE,TXN_HR,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,TRANSACTION_AMOUNT,CC_UTILISED_PRCNT,MCC,...,USER_BUREAU_STATUS,CREDIT_LIMIT,CC_AGE,TIME_BUCKET,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY,anamoly_scores,anomaly
144967,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-08,19,2,26.89,31,26.89,86.873810,5812,...,2,2100.0,13,1,1,26.89,1,0,-0.508667,1
144968,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-09,0,2,93.93,31,93.93,86.873810,5651,...,2,2100.0,13,0,1,93.93,1,2,-0.521941,1
144969,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-09,6,1,1703.53,31,1703.53,87.793333,3026,...,2,2100.0,13,0,2,1797.46,2,2,-0.672711,-1
144970,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-10,17,2,5.00,31,5.00,87.111905,7210,...,2,2100.0,13,1,1,5.00,1,3,-0.530777,1
144966,0002be66-d809-4ca1-a5a0-4a522adc9323,0,2022-04-11,3,1,14.31,31,14.31,87.793333,5815,...,2,2100.0,13,0,1,14.31,1,1,-0.416860,1


In [ ]:
df3['today'] = dt.date.today()

df3['today'] = pd.to_datetime(df3['today'])
df3['TXN_DATE'] = pd.to_datetime(df3['TXN_DATE'])
df3['day_diff'] = df3['today'] - df3['TXN_DATE']
last_day_txn = df3[df3['day_diff']== '1 days']
df3.drop(['day_diff','today'], axis=1, inplace=True)
last_day_txn.drop(['day_diff','today'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
len(last_day_txn)

4981

In [ ]:
anomaly_data = last_day_txn[last_day_txn['anomaly']==-1]
anomaly_data.head()

,USER_ID,USER_PROFILE_TYPE,TXN_DATE,TXN_HR,CARD_PROGRAM,REQUESTED_AMOUNT,REQUESTED_AMOUNT_CURRENCY,TRANSACTION_AMOUNT,CC_UTILISED_PRCNT,MCC,...,USER_BUREAU_STATUS,CREDIT_LIMIT,CC_AGE,TIME_BUCKET,COUNT_24HR,AMT_24HR,COUNT_TIME_BUCKET,WEEK_DAY,anamoly_scores,anomaly
352847,01f8860a-20f3-49ff-813d-0ed707a039aa,1,2022-04-18,9,1,314.93,31,314.93,10.911806,3009,...,2,7200.0,21,2,3,335.91,3,1,-0.624690,-1
3585,04c616d5-c11e-4337-9bcd-fb34ff541743,0,2022-04-18,20,1,2098.00,31,2098.00,83.613800,5732,...,2,5000.0,237,1,1,2098.00,1,1,-0.652941,-1
5983,08b1c2b1-9a6f-4c3a-badd-9e41b736389e,0,2022-04-18,21,1,1721.43,31,1721.43,92.788182,3026,...,0,2200.0,223,1,1,1721.43,1,1,-0.665438,-1
445422,09febe03-d676-4857-96e1-647f1c1eb66d,1,2022-04-18,21,2,1006.71,31,1006.71,14.381571,5732,...,2,7000.0,5,1,1,1006.71,1,1,-0.685092,-1
434139,1981480c-14cb-4eec-9d81-dff05993c1af,0,2022-04-18,1,1,1352.67,31,1352.67,98.967200,5942,...,0,2500.0,57,0,2,2047.51,2,1,-0.658410,-1


In [ ]:
len(anomaly_data)

54

In [ ]:
len(anomaly_data['USER_ID'].unique())

50

In [ ]:
org_txn_id = anomaly_data['ORIGINAL_TRANSACTION_ID'].unique()